In [ ]:
# !pip3 install --upgrade google-genai
# !pip install vertexai
# pip install google-generativeai google-auth
# gcloud auth application-default login

  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.18.0
    Uninstalling google-genai-1.18.0:
      Successfully uninstalled google-genai-1.18.0



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(r"C:/Users/janbe/Documents/GitHub Repos/pulse/secrets/.env", override=True)

# Retrieve project ID and location from environment variables
project_id = os.getenv("GOOGLE_CLOUD_PROJECT")
location = os.getenv("GOOGLE_CLOUD_LOCATION")
# GOOGLE_APPLICATION_CREDENTIALS will be picked up automatically by the SDK
# GOOGLE_GENAI_USE_VERTEXAI=True also configures the SDK

if not project_id or not location:
    raise ValueError(
        "GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_LOCATION must be set in your .env file."
    )
if not os.getenv("GOOGLE_APPLICATION_CREDENTIALS"):
    raise ValueError(
        "GOOGLE_APPLICATION_CREDENTIALS must be set in your .env file to the path of your service account key."
    )

In [47]:
import vertexai
from vertexai.generative_models import (
    GenerativeModel,
    Part,
    HarmCategory,
    HarmBlockThreshold,
)


def generate(prompt: str):
    """Generates text using Gemini 1.5 Flash model on Vertex AI."""

    print(f"Initializing Vertex AI for project: {project_id}, location: {location}")
    # Initialize Vertex AI. The SDK will automatically use the credentials from GOOGLE_APPLICATION_CREDENTIALS.
    vertexai.init(project=project_id, location=location)

    # Specify the Gemini 1.5 Flash model
    model = GenerativeModel("gemini-2.0-flash-001")

    print(f"Sending prompt to model: {prompt}")
    response = model.generate_content(
        contents=[prompt],
        generation_config={
            "max_output_tokens": 8192,  # Max tokens for 1.5 Flash
            "temperature": 0.9,
            "top_p": 1.0,
            "top_k": 32,
        },
        safety_settings={
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        },
    )
    print("Inference successful!")
    return response.text

In [48]:
user_prompt = "Explain the concept of quantum entanglement in simple terms."
generated_text = generate(user_prompt)
print("\n--- Generated Response ---")
print(generated_text)

Initializing Vertex AI for project: pulse-462613, location: europe-central2
Sending prompt to model: Explain the concept of quantum entanglement in simple terms.
Inference successful!

--- Generated Response ---
Imagine you have two coins. Instead of flipping them normally, you have a special machine that *always* produces one coin that's heads and one that's tails. You put each coin in a separate box and send one box to your friend Alice on the moon and keep the other.

Before you open your box, you don't know if you have heads or tails. Alice doesn't know either.

But, as soon as you open your box and see that you have heads, you instantly know that Alice *must* have tails, even before she opens her box! It's like the coins are linked, or *entangled*, in a way that their fates are tied together.

**Quantum entanglement is similar, but weirder! Here's the breakdown:**

*   **Tiny Particles:** Instead of coins, we're talking about tiny particles like electrons or photons.

*   **Proper

In [55]:
usage_metadata = response.usage_metadata
if usage_metadata:
    print(f"Input tokens (from response): {usage_metadata.prompt_token_count}")
    print(f"Output tokens (from response): {usage_metadata.total_token_count - usage_metadata.prompt_token_count}")
else:
    print("Usage metadata not available in the response.")


Input tokens (from response): 8
Output tokens (from response): 795


In [57]:
usage_metadata.thoughts_token_count

786